In [41]:
import ee
import numpy as np
import os
import palettable 
import numpy as np
from Satellite import *
from Simulator import *

service_account = "jims-948@jimssatellites.iam.gserviceaccount.com"
key_path = 'jimssatellites-25b0240d5cf2.json'
credentials = ee.ServiceAccountCredentials(service_account, key_path)
ee.Initialize(credentials=credentials)
import geemap
import folium



sat = Satellite("sat1.txt", "ISS")
sim = Simulator([sat], [])
sim.simulate(0,3*60*60, 1, motionEquation)
mappingArray= []


Simulating : 100%|█████████▉| 99.99999999997506/100 [06:50<00:00,  4.11s/it]   


In [40]:

count = 0
for image in  sat.payload.images:
    coord1 = list(image[0][0:2])
    coord2 = list(image[1][0:2])
    coord3 = list(image[2][0:2])
    coord4 = list(image[3][0:2])
    coord1.reverse()
    coord2.reverse()
    coord3.reverse()
    coord4.reverse()
    bounding = [coord1[0], coord1[1],  coord4[0], coord4[1]]

    roi = ee.Geometry.Rectangle(bounding)
    newFeature = ee.Feature(roi, {'name': str(count)})

    count = count+1
    mappingArray.append(newFeature)

# Create an interactive map
Map = geemap.Map(center=[0, 0], zoom=2)


#Create region of interest
roi = ee.Geometry.Rectangle([ 146,-35.36,  158.6, -30.83])
roi2 = ee.Geometry.Rectangle([ 150,-35.36,  178.6, -30.83])
roi3 = ee.Geometry.Polygon(
        [[120, -35.36],
         [150, 0],
         [170, 0],
         [146, 5]])

#I have no idea why, but for some reason this just paints everything so I'm gonna cop for now
mask = ee.Image(0).toInt8().paint(roi, 1)


# Create a feature collection
fc = ee.FeatureCollection([
    # Create features using ee.Geometry.Point, ee.Geometry.Polygon, etc.
    ee.Feature(roi, {'name': 'Mapped Area'}),
    ee.Feature(roi2, {'name': 'Mapped Area 2'}),
    ee.Feature(roi3, {'name': 'Mapped Area 3'})
    
])

fc = ee.FeatureCollection(mappingArray)

# Fill and outline the polygons in two colors
filled = mask.paint(fc, 2)
both = filled.paint(fc, 1, 5)
# Mask off everything that matches the fill color.
result = both.mask(filled.neq(2))


out_dir = os.path.expanduser('~/Downloads/GEEBAM')

GB = os.path.join(out_dir, 'GEEBAM_v3p1_dNBR_Classes.tif')

# Add layer to map

Map.add_raster(GB, band =1, palette = 'inferno',  layer_name = "GEEBAM")
Map.addLayer(result, {
    'palette': '000000,FF0000',
    'max': 1,
    'opacity': 0.5
}, 'Opaque White Rectangle')

Map


The provided color (000000,FF0000) is invalid. Using the default black color.
'#000000,FF0000' is not in web format. Need 3 or 6 hex digit.


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…